## Introdução

Existem outras bibliotecas que facilitam web scraping, como requests e bs4, retornando o html das páginas web. Porém, nesse caso em específico, as perguntas e respostas contidas nos textos não seguiam um padrão bem determinado, seja em classes ou na sua formatação. Por isso, decidi que seria menos oneroso e mais eficiente realizar esse web scraping por um método um pouco mais "manual". 


# Importação das bibliotecas


In [28]:
#importação das bibliotecas necessárias
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import pandas as pd

# Urls e partes importantes dos textos

In [29]:
#lista de urls de perguntas e respostas do goias.gov
urls = ["https://goias.gov.br/social/perguntas-e-respostas-frequentes-aprendiz-do-futuro/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-dignidade/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-maes-de-goias/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-goias-por-elas/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-sistema-socioeducativo/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-auxilios-agua-e-energia-pao-e-leite/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-carteira-de-identificacao-do-autista2/",
        "https://goias.gov.br/social/bolsa-familia/",
        "https://goias.gov.br/social/cadastro-unico-cadunico/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passaporte-intermunicipal-da-pessoa-idosa/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passe-livre-intermunicipal-da-pessoa-com-deficiencia/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-centro-de-referencia-estadual-da-igualdade-crei/"]

#Antes de cada primeira pergunta de cada url, sempre irá aparecer o texto descrito na variável "inicio"
inicio = """Categoria  

Perguntas e Respostas Frequentes"""

#No final da última resposta de cada url, sempre irá aparecer o texto descrito na variável "fim"
fim = """


 











 









 













 

 





 Governo na palma da mão"""

#precisamos desse complemento do texto para conseguir capturar as perguntas que contém um ponto final no meio da frase, como por exemplo: 
#Fui vítima de racismo. O que devo fazer?
complemento_texto = "Fui vítima de"

# Criação das listas e extração dos textos

In [30]:
#inicializar as listas que iremos precisar para extrair os textos das urls utilizando a langchain
textos_brutos = []
textos_conteudo = []
textos_process = []

#extrair somente os textos das urls
for url in urls:
    textos = WebBaseLoader(url)
    textos_brutos = textos.load()
    textos_conteudo.append(textos_brutos[0].page_content)

# Extração apenas das perguntas e respostas

In [31]:
#determinar o tamanho das variáveis de texto definidas acima para realizações de operações futuras
tamanho_inicio = len(inicio)
tamanho_final = len(fim)
tamanho_complemento = len(complemento_texto)

#atribuir a lista texto_process apenas as perguntas e respostas do texto
for respostas_ind in textos_conteudo:
    for j in range(len(respostas_ind) - tamanho_inicio):
        if inicio in respostas_ind[j:j + tamanho_inicio]:
            começo_frase = j + tamanho_inicio
        elif fim in respostas_ind[j:j+tamanho_final]:
            textos_process.append(respostas_ind[começo_frase + 6 : j])

# Inicialização das listas

In [32]:
#inicializando listas para operações futuras
textos_split = []
questions = []
perguntas = []
answers = []
respostas = []

# Extração das perguntas a partir do texto

In [33]:
#Iterando cada texto da lista de textos_process
for texto in textos_process:
    #Iterando sobre cada caractere e seu respectivo indice do texto
    for i, caractere in enumerate(texto):
        aux = i 
        #Procura um ponto de interrogação nos caracteres do texto
        if "?" in caractere:
            #Inicializa a variável pergunta com uma string vazia
            pergunta = ""
            #Inicializa o cont
            cont = 1
            #Inicializa o contador
            contador = 0
            #Permanecer no loop até que contador seja diferente de 1
            while contador != 1:
                #Se o texto de indice i - cont conter algum dos seguintes caracteres: "." ou ";" ou "\n", a variável j irá receber i - cont e 
                #contador recebe 1 para encerrar o looping. Isso é necessário pois o indice i está indicando o caractere "?", ou seja, o final
                #de uma pergunta. Então precisamos varrer a string em busca do começo dela. Antes de todo inicio de pergunta, há o ponto final
                #da resposta da pergunta anterior (questions[i - 1]). Em alguns casos isolados no texto, o final da pergunta é determinado por ;
                #ou por \n, sendo o último para as primeiras perguntas que cada url
                if texto[i - cont] == "." or texto[i - cont] == ";" or texto[i - cont] == "\n" or (i - cont) == 0 :
                    j = i - cont
                    contador = 1
                #Caso não, cont é incrementado em 1 para continuar a procura pelos caracteres ".", ";" ou "\n"
                else:
                    cont = cont + 1
            #Variável cont é resetado
            cont = 0
            #Permanecer no looping enquanto j(que atribuimos o valor de i - cont) + cont for menor ou igual a i
            while j + cont <= i:
                #A variável pergunta recebe pergunta (na primeira iteração está com string vazia) + o texto[j + cont], obtendo a pergunta inteira
                pergunta = pergunta + texto[j + cont]
                #Incrementa 1 à variável cont
                cont = cont + 1
            #Se a pergunta for do caso especial especificado abaixo, onde há . na pergunta, passará por esse if
            if pergunta == ". O que devo fazer?":
                #Auxiliar recebe i
                aux = i
                #Enquanto o complemento_texto, ou seja "Fui vítima de" não estiver no texto de auxiliar até auxiliar mais tamanho do complemento_texto
                #o looping continua
                while complemento_texto not in texto[aux : aux + len(complemento_texto)]:
                    #Auxiliar sofre um decremento de 1 para continuar a procura
                    aux = aux - 1
                #No final, quando o complemento_texto for detectado no intervalo especificado, o looping irá parar, então atribuímos
                #a variável pergunta o texto de auxiliar até i + 1 (para incluir o ponto final)
                pergunta = texto[aux : i + 1]
            #Por fim, é realizado um append de pergunta na lista questions para armazenar cada pergunta
            questions.append(pergunta)

# Limpeza dos dados na lista de perguntas

In [34]:
#Percorremos cada indice e cada string da lista de perguntas
for i, question in enumerate(questions):
    #Se a pergunta for 'O que é o CREI?', a pergunta é deletada. Isso ocorre pois há dois pontos de interrogação na pergunta, fazendo com que
    #nosso algoritmo acima interprete como duas perguntas
    if question == 'O que é o CREI?':
        del questions[i]
#Usamos a função replace para a retirada dos espaços em branco das questões e substituímos por string vazia
questions = [caractere.replace("\n", "") for caractere in questions]

# Extração das respostas a partir do texto

In [35]:
#Resetando o cont
cont = 0
#Enquanto o cont for menor do que o tamanho da lista de questões
while cont < len(questions):
    #Iterando cada texto da lista de textos_process
    for texto in textos_process:
        #Iterando sobre cada caractere e seu respectivo indice do texto
        for i, caractere in enumerate(texto):
            #Enquanto cont for menor que o tamanho da lista de questões - 1
            if cont < len(questions) - 1:
                #Se o questions[cont] (na primeira iteração, questions [0]) está presente o texto[i até i + tamanho da questão]
                if questions[cont] in texto[i: i + len(questions[cont])]:
                    #Inicio_respostas recebe o último caractere da pergunta, ou seja "?", + 1 para receber o inicio das respostas
                    inicio_respostas = i + len(questions[cont]) + 1
                    #Se cont + 1 for menor que o tamanho da lista de questões
                    if cont + 1 < len(questions):
                        #j recebe inicio_respostas pois o valor de inicio_respostas não pode ser perdido
                        j = inicio_respostas
                        #j irá iterar pelo tamanho do texto - tamanho da próxima pergunta
                        for j in range (len(texto) - len(questions[cont + 1])):
                            #Se a próxima pergunta estiver continha de j até j + o tamanho da próxima pergunta
                            if questions[cont + 1] in texto[j: j + len(questions[cont + 1])]:
                                #Então é realizado um append de inicio_respostas até j - 1, isso é feito para não adicionar o primeiro caractere
                                #da pergunta, na lista answers para armazenar cada resposta
                                answers.append(texto[inicio_respostas : j - 1])
                                #cont recebe incremento de 1
                                cont = cont + 1
                                break
                            #Se a próxima questão não está no texto e o tamanho de j + o tamanho da próxima questão for == ao tamanho do texto
                            #Isso é necessário para conseguir armazenar a última resposta da última pergunta de cada url
                            if questions[cont + 1] not in texto[j: j + len(questions[cont + 1])] and j + len(questions[cont + 1]) == len(texto) - 1:
                                 #Então é realizado um append de i + o tamanho da questão atual na lista answers para armazenar cada resposta
                                answers.append(texto[i + len(questions[cont]) :])
                                cont = cont + 1
                                break
            #Se for a última pergunta
            elif cont == len(questions) - 1:
                #É realizado um append de i até o final do texto, pois todo o restante faz parte da resposta
                answers.append(texto[i:])
                #Cont é incrimentado em 1 em o looping é encerrado
                cont = cont + 1

# Limpeza dos dados na lista de respostas

In [36]:
#A função replace é utilizada para trocar o "\n" e o "_ " por espaço em branco 
answers = [caractere.replace("\n", "") for caractere in answers]
answers = [caractere.replace("– ", "") for caractere in answers]

# Armazenando as perguntas e respostas em apenas uma lista

In [38]:
#É criada a lista de all_data para armazenar tanto as perguntas como as respostas
all_data =[]
#Item iterrando no tamanho da lista questions (ambas as listas tem o mesmo tamanho)
for item in range(len(questions)):
    question = questions[item]
    answer = answers[item]
    #É realizado um append das perguntas e respostas para a lista all_data
    all_data.append({'question': question, 'answer': answer})

# Criando um arquivo .csv contendo as perguntas e respostas

In [39]:
#A lista all_data é transformada em um objeto dataframe
data_df = pd.DataFrame(all_data)
#É criado um arquivo .csv contendo as perguntas e as respostas no diretório /Data
data_df.to_csv('Data/rag_doc.csv', index=False)